In [1]:
#Dependencies

import requests
import pandas as pd
import json

In [ ]:
#Only make API calls that were needed based on list of cities from happiness score data


#Create a list of cities to use to make API call
#import happiness data csv

import_happiness_states = pd.read_csv("Happiness.csv", encoding='ansi')

#need pandas .copy function to create an unchanging version of the original
df_cities = import_happiness_states.copy()

#df.drop(['Overall Rank', 'Total Score', 'Emotional & Physical Well-Being','Income & Employment', 'Community & Environment'], axis=1)
df_cities.drop(df_cities.columns[[0, 2, 3,4,5]], axis=1, inplace=True)

hplist = df_cities['City'].tolist()
print(hplist)





In [6]:
#Prepare list of Happiness Cities for loop by dropping the comma and state

updated_hplist = []
for x in range(len(hplist)):
    
   
    updated_hplist.append(hplist[x][:-4])

#test output
# print(updated_hplist)

In [7]:
#API call to get a list of cities


url = "https://zylalabs.com/api/226/cities+cost+of+living+and+average+prices+api/232/get+cities"
    
headers = {
    'Authorization': 'Bearer 326|qySQCd3uZ7I39Bh8v1680nDrxjlB0liuk1zvdCXv'
}

#data is the list of cities
    
data = requests.request("GET", url, headers=headers).json()




In [ ]:
#Cost of living Cities

#compare the list of Happiness cites to cost of living cities
#if CoL city (from API call) is inside of Happiness cities (updated_hplist) append to matching cities
#then do length of matching cities list to see how many matches

matching_cities = []

for x in range(len(data['cities'])):
    
  if data['cities'][x]['country_name'] == "United States":
            
    current_city = (data['cities'][x]['city_name'] + ", " + data['cities'][x]['state_code'])

    if current_city in updated_hplist:
      
      matching_cities.append(current_city)
      
#Check number of cities to make sure there are enough matches
print(len(matching_cities))

#Test response
# if (response_data['cities'][0]['city_name']) == "Herat":
#     print("Herat is in the list")

In [ ]:
#compare matching cities in the API with cities from the happiness database and make a list of missing cities

missing_cities = []

for x in hplist:
    if x not in matching_cities:
           missing_cities.append(x)

print(missing_cities)

In [ ]:
#Test response from API
# if (response_data['cities'][0]['city_name']) == "Herat":
#     print("Herat is in the list")

#Test happiness Data response
# df_cities['State'] = df_cities['City'].str[-2:]
# df_cities['City'] = df_cities['City'].str[:-4]

# df_cities['State']


In [ ]:
#Use API call to get city data and store in a json

#create empty lists to fill with appended data
api_city = []
api_state = []
cities = []
#more here

#change index values to loop through entire column of cities when ready
for city in df_cities['City']:
    #print(city)
    try:
    
        city_no_spaces = city.replace(" ", "+")
        url = "https://zylalabs.com/api/226/cities+cost+of+living+and+average+prices+api/233/get+prices?city_id=2294&city_name="+city_no_spaces+"&country_name=united+states"
        #print(url)
        headers = {'Authorization': 'Bearer 476|T9c5gji6ogKFXaEZxz8ChZTpaON6VJ2ejSzGG8aQ'}
                    
        response = requests.request("GET", url, headers=headers).json()
        r = response
        # api_city.append(response['city_name'])
        # api_state.append(response['state_code'])
        cities.append(r)

    except Exception as e:
        print(e)
        continue
    #pull the cost data here and append to empty lists

In [ ]:
#Save API data locally

df_fullcoldata = pd.DataFrame(cities)

df_fullcoldata.to_csv('fullcoldata.csv')


In [ ]:
print(cities[0]['state_code'])

In [ ]:
#Create column names as feature list for the final dataframe
#Started with State to ensure first column is state
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.tolist.html

df_prices = pd.DataFrame(r['prices']) 



featureslist = ['State']

for x in df_prices['item_name']:
    featureslist.append(x)
#feature_list

print(featureslist)

In [ ]:
#Use a loop to Pull all averages value from API response json and store in a dictionary 

#city_averages = []
city_avgs_dict = {}

for city_object in cities:

    city_avgs = []
    
    try:
        city = city_object['city_name']
        state = city_object['state_code']
        price_object = (city_object['prices'])

    except Exception as e:
       continue
        # print(city_object)
        # print(e)
    finally:

#create dataframe of city and average prices of select objects
        city_avgs.append(state)

        for x in range(len(price_object)):
            average = price_object[x]['avg']
            city_avgs.append(average)

        city_avgs_dict[city] = city_avgs
            
print(city_avgs_dict)



In [ ]:
df = pd.DataFrame.from_dict(city_avgs_dict, orient='index', columns=featureslist)


df_coledit = df.copy()

#export to csv to clean column names

df_coledit.to_csv('coldata_full_.csv')


In [ ]:
#import csv with cleaned column names
df_col = pd.read_csv('coldata_full_.csv')

#need pandas .copy function to create an unchanging version of the original
df_coledit = df_col.copy()

df_coledit.keys()

In [ ]:
#clean the data the cost of living csv
#App is unstable and giving errors. support aware but completed fix unknown
#converted wrong values and nulls to median

median_value = df_coledit['Chicken Breasts (1 kg)'].median()

df_coledit.loc[df_coledit['Chicken Breasts (1 kg)'] > 15.43, 'Chicken Breasts (1 kg)' ] = median_value

median_valuegb = df_coledit['Ground Beef, 1 kg'].median()

df_coledit.loc[df_coledit['Ground Beef, 1 kg'] > 12, 'Ground Beef, 1 kg' ] = median_valuegb

median_valuebw = df_coledit['Bottle of Wine'].median()

df_coledit.loc[df_coledit['Bottle of Wine'] > 18.5, 'Bottle of Wine' ] = median_valuebw

median_valueb = df_coledit['Banana, 1 kg'].median()

df_coledit.loc[df_coledit['Banana, 1 kg'] > 3.59, 'Banana, 1 kg' ] = median_valueb

median_valueeg = df_coledit['Eggs, 12 pack'].median()

df_coledit.loc[df_coledit['Eggs, 12 pack'] > 13.22, 'Eggs, 12 pack' ] = median_valueeg

median_valuelt = df_coledit['Lettuce, 1 head'].median()

df_coledit.loc[df_coledit['Lettuce, 1 head'] > 9.5, 'Lettuce, 1 head' ] = median_valuelt

median_valuebd = df_coledit['White Bread (loaf)'].median()

df_coledit.loc[df_coledit['White Bread (loaf)'] > 10, 'White Bread (loaf)' ] = median_valuebd

median_valuecz = df_coledit['Local Cheese, 1 kg'].median()

df_coledit.loc[df_coledit['Local Cheese, 1 kg'] > 14.7, 'Local Cheese, 1 kg' ] = median_valuecz

median_valueo = df_coledit['Onion, 1 kg'].median()

df_coledit.loc[df_coledit['Onion, 1 kg'] > 9, 'Onion, 1 kg' ] = median_valueo

median_valuemk = df_coledit['Milk, 1 liter'].median()

df_coledit.loc[df_coledit['Milk, 1 liter'] > 8.39, 'Milk, 1 liter' ] = median_valuemk

median_valueor = df_coledit['Oranges, 1 kg'].median()

df_coledit.loc[df_coledit['Oranges, 1 kg'] > 10, 'Oranges, 1 kg' ] = median_valueor


median_valuepo = df_coledit['Potato, 1 kg'].median()

df_coledit.loc[df_coledit['Potato, 1 kg'] > 5.64, 'Potato, 1 kg' ] = median_valuepo

median_valuerc = df_coledit['White Rice, 1 kg'].median()

df_coledit.loc[df_coledit['White Rice, 1 kg'] > 11, 'White Rice, 1 kg' ] = median_valuerc

median_valueto = df_coledit['Tomato, 1 kg'].median()

df_coledit.loc[df_coledit['Tomato, 1 kg'] > 7.34, 'Tomato, 1 kg' ] = median_valueto

median_valuew = df_coledit['Water, 1.5 liter'].median()

df_coledit.loc[df_coledit['Water, 1.5 liter'] > 4.75, 'Water, 1.5 liter' ] = median_valuew

median_valuew = df_coledit['Water, 1.5 liter'].median()

df_coledit.loc[df_coledit['Water, 1.5 liter'] < .57, 'Water, 1.5 liter' ] = median_valuew




median_value1bd = df_coledit['One bedroom apartment '].median()

df_coledit.loc[df_coledit['One bedroom apartment '] < 475, 'One bedroom apartment ' ] = median_value1bd

median_value1bd = df_coledit['One bedroom apartment '].median()

df_coledit.loc[df_coledit['One bedroom apartment '] > 2695.6, 'One bedroom apartment ' ] = median_value1bd

median_value3bd = df_coledit['Three bedroom apartment '].median()

df_coledit.loc[df_coledit['Three bedroom apartment '] > 4524.14, 'Three bedroom apartment ' ] = median_value3bd

median_value3bd = df_coledit['Three bedroom apartment '].median()

df_coledit.loc[df_coledit['Three bedroom apartment '] < 850, 'Three bedroom apartment ' ] = median_value3bd

median_valueut = df_coledit['Basic utilities (Electricity, Heating or Cooling, Water and Garbage)'].median()

df_coledit.loc[df_coledit['Basic utilities (Electricity, Heating or Cooling, Water and Garbage)'] < 103.28, 'Basic utilities (Electricity, Heating or Cooling, Water and Garbage)' ] = median_valueut

df_coledit['Basic utilities (Electricity, Heating or Cooling, Water and Garbage)'].fillna(value=median_valueut, inplace=True)

median_valueweb = df_coledit['Internet, 60 Mbps or More, Unlimited Data, Cable/ADSL'].median()

df_coledit.loc[df_coledit['Internet, 60 Mbps or More, Unlimited Data, Cable/ADSL'] < 24, 'Internet, 60 Mbps or More, Unlimited Data, Cable/ADSL' ] = median_valueweb

df_coledit['Internet, 60 Mbps or More, Unlimited Data, Cable/ADSL'].fillna(value=median_valueweb, inplace=True)


In [ ]:
df_coledit.to_csv('coldata_full_final.csv')

In [17]:
#Going forward, finding an API that is stable.  API is not stable and gives too many errors
#Possible to web scrape the Numbeo cost of living website
# Bureau of Labor Statistics data may work here